In [1]:
import random
import matplotlib.pyplot as plt
from pprint import pprint

from pennylane import numpy as np
import pennylane as qml
from pennylane.templates import AngleEmbedding, AmplitudeEmbedding, BasisEmbedding
from pennylane.operation import Tensor
#https://pennylane.readthedocs.io/en/stable/introduction/templates.html

In [2]:
dev1 = qml.device("default.qubit", wires=2)

In [3]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    return qml.expval(qml.PauliZ(0)) #0 or 1: Z, + or -: X, +i or -i: Y

## Ansatz

In [96]:
n_qubits = 4
dev2 = qml.device("default.qubit", wires=n_qubits)

def ansatz0():
    if (random.randint(0, 5) > 2):
        qml.RX(np.pi/2, wires=0)
    if (random.randint(0, 10) > 5):
        qml.RX(np.pi/2, wires=2)

def ansatz1():
    #Qubits 1, 2
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 0])
    qml.Hadamard(wires=0)
    #Qubits 3, 4
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[3, 2])
    qml.Hadamard(wires=2)
    
def ansatz2():
    #Qubits 1, 2
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 0])
    qml.Hadamard(wires=0)
    
def ansatz3():
    #Qubits 3, 4
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)
    qml.CNOT(wires=[3, 2])
    qml.Hadamard(wires=2)

    

def ansatzQFT():
    qml.QFT(wires=[0, 1, 2, 3])



@qml.qnode(dev2)
def circuitMix():
    ansatz0()
    ansatz1()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


@qml.qnode(dev2)
def circuitShake():
    ansatz0()
    if (random.randint(0, 20) > 10):
        ansatz2()
    else:
        ansatz3()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


@qml.qnode(dev2)
def circuitQFT():
    qml.RX(np.pi/2, wires=0)
    qml.RX(np.pi/2, wires=2)
    ansatzQFT()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [97]:
def rounder(x):
    y = []
    for i in x:
        if i < -0.9:
            y.append(-1)
        elif i > -0.1 and i < 0.1:
            y.append(0)
        elif i > 0.9:
            y.append(1)
        else:
            y.append(i)
    return y

In [98]:
print(rounder(circuitMix()))

[0, 0, 1, 0]


In [99]:
print(rounder(circuitShake()))

[1, 0, 1, 1]


In [100]:
print(rounder(circuitQFT()))

[0, tensor(0.60355339, requires_grad=True), 0, 0]
